<h1 style="font-size:30px;">Trénink hluboké konvoluční GAN</h1>

V tomto notebooku budeme trénovat Deep Convolutional Generative Adversarial Neural Network (DCGAN) na komplexních RGB obrázcích. Obecný koncept a model GAN jsou v podstatě stejné jako v předchozím notebooku. Jediný skutečný rozdíl je v tom, že sítě v tomto notebooku používají konvoluční vrstvy místo plně propojených vrstev. A budeme používat větší barevné obrázky místo menších černobílých obrázků. V důsledku toho bude čas potřebný k natrénování modelu mnohem delší.

<img src="https://opencv.org/wp-content/uploads/2022/09/c4-gan-dcgan-architecture.png" width=900>

## Table of Contents

* [1 Úvod do DCGAN a proč je potřebujeme](#1-Úvod-do-DCGAN-a-proč-je-potřebujeme)
* [2 Konfigurace systému](#2-Konfigurace-systému)
* [3 Konfigurace tréninku a datové sady](#3-Konfigurace-tréninku-a-datové-sady)
* [4 Připravte datovou sadu](#4-Připravte-datovou-sadu)
* [5 Síť Generátoru](#5-Síť-Generátoru)
* [6 Síť Diskriminátoru](#6-Síť-Diskriminátoru)
* [7 Definujeme ztrátovou funkci](#7-Definujeme-ztrátovou-funkci)
* [8 Definujeme optimalizátory](#8-Definujeme-optimalizátory)
* [9 Ukládání kontrolních bodů a vytváření adresářů](#9-Ukládání-kontrolních-bodů-a-vytváření-adresářů)
* [10 Trénink](#10-Trénink)
* [11 Zobrazení grafů výsledků tréninku](#11-Zobrazení-grafů-výsledků-tréninku)
* [12 Create Images using the Trained Model](#12-Create-Images-using-the-Trained-Model)
* [13 Conclusion](#13-Conclusion)

In [ ]:
import os
import time
import numpy as np
import torch
import os
import matplotlib.pyplot as plt
import cv2
import requests
import zipfile
import torch.nn.functional as F

from dataclasses import dataclass
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter)
from matplotlib import gridspec
from tqdm.auto import tqdm
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import (
    nn,
    optim
)

block_plot = False

## 1 Úvod do DCGAN a proč je potřebujeme

V předchozím notebooku jsme trénovali základní GAN (složený pouze z plně propojených vrstev). Použili jsme datovou sadu Fashion MNIST, která obsahuje černobílé obrázky. Nicméně, základní GANy mají svá omezení. Nefungují dobře na komplexních RGB (barevných) obrázcích. Jedním z důvodů je, že zplošťujeme všechny obrázky jak pro generátor, tak pro diskriminátor. To vede ke ztrátě prostorových informací.

K vyřešení tohoto problému máme Hluboké Konvoluční Generativní Adversářské Sítě (zkráceně DCGAN). Jak název napovídá, architektura DCGAN se skládá z konvolučních vrstev. Již známe sílu konvolučních vrstev při zachycování rysů v obrázcích. DCGAN byly poprvé představeny v článku <a href="https://arxiv.org/pdf/1511.06434v2.pdf" target="_blank">Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks</a> od Aleca Radforda a kol.

### 1.1 Generátor
Pojďme se podívat na generátorovou síť

<img src="https://opencv.org/wp-content/uploads/2022/09/c4-gan-dcgan-architecture.png">

Stejně jako dříve, síť generátoru síť vyžaduje jako vstup latentní vektor šumu. Zbytek sítě se skládá z konvolučních vrstev, kde každá následující vrstva snižuje počet výstupních kanálů od počátečních 1024 až na 3, protože síť musí generovat RGB barevné obrázky (3 kanály). Všimněte si také, že prostorová dimenze v každé vrstvě se neustále zvětšuje, dokud konečná vrstva nemá prostorový tvar $64\times64$. To znamená, že generátor produkuje obrázky s rozlišením 64x64x3 v konečné vrstvě. Vzpomeňte si z modulu Segmentace, že takovéto vzorkování  nahoru se provádí pomocí transponovaných konvolucí.

### 1.2 Diskriminátor

Stejně jako dříve je síť diskriminantu binární klasifikátor, ale v tomto případě budeme používat konvoluční vrstvy podobně jako všechny ostatní klasifikátory CNN, které jsme si v kurzu ukázali. Existuje však několik věcí, které se trochu liší od ostatních CNN, které jsme použili. Například v diskriminátoru nepoužíváme žádné 2D Max-Pooling vrstvy a používáme postupnou konvoluci (krok > 1), abychom stále zmenšovali velikosti map prvků.


### 1.3 Architektonické zásady pro stabilní trénování DCGAN

Existuje několik důležitých zásad, které autoři doporučují po rozsáhlých experimentech s architekturami DCGAN. Tyto zásady budeme také dodržovat v tomto notebooku při budování architektur generátoru a diskriminátoru.

<img src="https://opencv.org/wp-content/uploads/2022/09/c4-gan-dcgan-training-guideline.png" width=650>



## 2 Konfigurace systému

In [ ]:
def set_seed(SEED_VALUE):
    torch.manual_seed(SEED_VALUE)
    torch.cuda.manual_seed(SEED_VALUE)
    torch.cuda.manual_seed_all(SEED_VALUE)
    np.random.seed(SEED_VALUE)

seed = 7
set_seed(seed)

## 3 Konfigurace tréninku a datové sady

Původní článek o DCGAN používá 100-dimenzionální vektor šumu. Nicméně z našich experimentů jsme zjistili, že mírně vyšší dimenzionální vektor šumu (`LATENT_DIM: int = 128`) funguje lépe na komplexních datových sadách, jako je ta, kterou zde použijeme.

In [ ]:
@dataclass(frozen=True)
class TrainingConfig:
    DEVICE: torch.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    NUM_EPOCHS: int = 31
    LATENT_DIM: int = 128
    BATCH_SIZE: int = 16
    LEARNING_RATE_D: float = 0.0002
    LEARNING_RATE_G: float = 0.0002
    CHECKPOINT_DIR: str = os.path.join('/kaggle/working/model_checkpoint', 'dcgan_flickr_faces')

class DatasetConfig:
    IMG_HEIGHT:   int = 64
    IMG_WIDTH:    int = 64
    NUM_CHANNELS: int = 3

## 4 Připravte datovou sadu

Použijeme zde datovou sadu Flickr Faces, která obsahuje 70 000 obrázků tváří. Datovou sadu jsme získali z <a href="https://www.kaggle.com/datasets/xhlulu/flickrfaceshq-dataset-nvidia-resized-256px" target="_blank">Kaggle</a>. Obrázky tváří byly již převedeny na rozlišení 256x256 pixelů. Pro trénink GAN je však dále zmenšíme na rozlišení 64x64 pixelů.

**Poznámka**: Jedná se o rozsáhlou datovou sadu obsahující 70 000 obrázků (2 GB v rozsahu). Je mnohem lepší spustit notebook na jádru Kaggle nebo Google Colab s GPU. Můžete jej také spustit lokálně, pokud máte dostatečně výkonný GPU.

In [ ]:
def download_file(url, save_name):
    url = url
    if not os.path.exists(save_name):
        file = requests.get(url)
        open(save_name, 'wb').write(file.content)

In [ ]:
def unzip(zip_file=None):
    try:
        with zipfile.ZipFile(zip_file) as z:
            z.extractall("/kaggle/working/dataset_flickr_faces")
            print("Extracted all")
    except:
        print("Invalid file")

In [ ]:
download_file(
    'https://www.dropbox.com/s/crg8qvtix3wg6kx/dataset_flickr_faces.zip?dl=1',
    '/kaggle/working/dataset_flickr_faces.zip'
)

unzip(zip_file='/kaggle/working/dataset_flickr_faces.zip')

Následující transformace `transform` převede obrázky na rozlišení 64x64, převede je na tenzory a normalizuje hodnoty pixelů do rozsahu [-1, 1].

In [ ]:
# Tranforms to resize and convert the pixels between -1 and 1.
transform = transforms.Compose([
    transforms.Resize((DatasetConfig.IMG_HEIGHT, DatasetConfig.IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

Zde můžeme použít třídu `ImageFolder` k snadnému vytvoření datové sady. To však vyžaduje, aby obrázky byly v subadresáři. Extrahujeme tedy obrázky do subadresáře `dataset_flickr_faces`.

In [ ]:
# Prepare the dataset.
dataset_train = datasets.ImageFolder(
    root='/kaggle/working/dataset_flickr_faces/',
    transform=transform
)

Následující blok kódu zobrazuje několik obrázků z datové sady.

In [ ]:
# Visualize a few images.
plt.figure(figsize=(20, 5))

for image in range(TrainingConfig.BATCH_SIZE):

    nrow = 2
    ncol = min(8, int(TrainingConfig.BATCH_SIZE/nrow))

    for i in range(nrow*ncol):
        image = dataset_train[i][0] / 2 + 0.5
        ax = plt.subplot(nrow, ncol, i + 1)
        plt.imshow(image.permute(1, 2, 0))
        plt.axis("off")

In [ ]:
# The data loader.
train_dataloader = DataLoader(
    dataset=dataset_train,
    batch_size=TrainingConfig.BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

## 5 Síť Generátoru


Při budování modelu generátoru se budeme snažit co nejvíce dodržet původní strukturu. 128-dimenzionální normální rozdělení je promítnuto do konvoluční reprezentace s malým prostorovým rozsahem s mnoha mapami rysů. Řada čtyř transponovaných konvolucí poté převede tuto reprezentaci vysoké úrovně na $64\times64$ pixelový obraz.

Začínáme s 1024-dimenzionální transponovanou konvoluční vrstvou a končíme s 64-dimenzionální vrstvou. Výstupní vrstva by měla generovat RGB obrázky. Má tedy 3 kanály. Všimněte si, že výstupy konečné transponované konvoluční vrstvy jsou přiváděny do aktivační funkce Tanh, která je důležitá pro generování dobrých výsledků.

V první transponované konvoluční vrstvě používáme velikost jádra (kernel size) 4 a krok (stride) 4. To má za následek mapu rysů $4\times4\times1024$. Ve všech následujících transponovaných konvolučních vrstvách používáme krok (stride) 2. To dvakrát zvětší mapu rysů v každém bloku. Výsledkem je, že model generátoru vyprodukuje v konečné vrstvě obrázek $64\times64\times3$.

Implementace generátoru zde je přesně stejná jako v původním článku o DCGAN, s výjimkou latentní dimenze. Jak jsme již dříve diskutovali, budeme používat 128-dimenzionální vektor šumu jako vstup do generátoru místo 100-dimenzionálního vektoru šumu.

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.nz = latent_dim
        self.generator = nn.Sequential(
            nn.ConvTranspose2d(
                self.nz, 1024, kernel_size=4,
                stride=4, padding=1
            ),
            nn.BatchNorm2d(1024),
            nn.ReLU(True),

            nn.ConvTranspose2d(
                1024, 512, kernel_size=4,
                stride=2, padding=1
            ),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(
                512, 256, kernel_size=4,
                stride=2, padding=1
            ),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(
                256, 128, kernel_size=4,
                stride=2, padding=1
            ),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(
                128, 64, kernel_size=4,
                stride=2, padding=1
            ),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(
                64, 3, kernel_size=4,
                stride=2, padding=1
            ),
            nn.Tanh()
        )

    def forward(self, input):
        return self.generator(input)

In [ ]:
generator = Generator(TrainingConfig.LATENT_DIM).to(TrainingConfig.DEVICE)
print(generator)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in generator.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in generator.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

## 6 Síť Diskriminátoru

Jak vidíte, máme poměrně velký generační model (více než 13 milionů parametrů), který následuje pokyny autorů. Budeme také potřebovat dostatečně velký diskriminační model, aby odpovídal kapacitě generátoru. Jinak bude generátor schopen velmi snadno oklamat diskriminátor a trénink GAN nemusí být úspěšný.

V původním článku o DCGAN autoři neposkytují žádné konkrétní výstupní dimenze pro 2D konvoluční vrstvy modelu diskriminátoru. Proto si je sami volíme, přičemž dodržujeme architektonické zásady navržené autory.

* Žádná vrstva normalizace dávek po vstupní vrstvě
* Použití Leaky ReLU namísto aktivační funkce ReLU

Model diskriminátoru má jednu normální konvoluční vrstvu následovanou třemi konvolučními vrstvami s krokem $2\times2$ pro snížení vzorku vstupního obrazu. Model nemá žádné vrstvy pooling a jediný uzel ve výstupní vrstvě s sigmoidní aktivační funkcí pro předpověď, zda je vstupní vzorek skutečný nebo falešný. Model je trénován k minimalizaci ztrátové funkce binární křížové entropie, vhodný pro binární klasifikaci. Použijeme některé osvědčené postupy při definování modelu diskriminátoru, jako je použití LeakyReLU namísto ReLU, použití Dropout a použití verze Adam stochastického gradientního sestupu s rychlostí učení 0.0002 a hybností 0.5. Třída `Discriminator()` níže definuje model diskriminátoru a parametrizuje velikost vstupního obrazu.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(
                3, 64, kernel_size=4, stride=2, padding=1
            ),
            nn.LeakyReLU(0.2),

            nn.Conv2d(
                64, 128, kernel_size=4, stride=2, padding=1
            ),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(
                128, 256, kernel_size=4, stride=2, padding=1
            ),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(
                256, 512, kernel_size=4, stride=2, padding=1
            ),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.Conv2d(
                512, 1024, kernel_size=4, stride=2, padding=1
            ),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2),

            nn.Conv2d(
                1024, 1, kernel_size=3, stride=4, padding=1
            )
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(1, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        bs, _, _, _ = x.shape
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        x = self.linear_layers(x)
        return x

In [ ]:
discriminator = Discriminator().to(TrainingConfig.DEVICE)
print(discriminator)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in discriminator.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in discriminator.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

## 7 Definujeme ztrátovou funkci

Zde použijeme stejnou ztrátovou funkci binární křížové entropie jako v předchozím notebooku.

In [ ]:
BCE = nn.BCELoss()

In [ ]:
def loss_func(y_true, y_pred):
    loss = BCE(y_pred, y_true)
    return loss

## 8 Definujeme optimalizátory

Zde použijeme pro obě sítě optimalizátor Adam a stejná nastavení optimalizátoru jako v předchozím notebooku, která jsou známa tím, že dobře fungují pro mnoho aplikací.

In [ ]:
optimizer_G = optim.Adam(
    generator.parameters(),
    lr=TrainingConfig.LEARNING_RATE_G,
    betas=(0.5, 0.999)
)
optimizer_D = optim.Adam(
    discriminator.parameters(),
    lr=TrainingConfig.LEARNING_RATE_D,
    betas=(0.5, 0.999)
)

## 9 Ukládání kontrolních bodů a vytváření adresářů

In [ ]:
if not os.path.exists(TrainingConfig.CHECKPOINT_DIR):
    os.makedirs(TrainingConfig.CHECKPOINT_DIR)

num_versions = len(os.listdir(TrainingConfig.CHECKPOINT_DIR)) + 1
ckpt_version_dir = TrainingConfig.CHECKPOINT_DIR + '/version_' + str(num_versions)
os.makedirs(ckpt_version_dir)

In [ ]:
checkpoint_dir = TrainingConfig.CHECKPOINT_DIR
generator_ckpt = os.path.join(ckpt_version_dir, 'generator'+'.pth')
discriminatr_ckpt = os.path.join(ckpt_version_dir, 'discriminator'+'.pth')

## 10 Trénink

V předchozím notebooku jsme vytvořili dvě samostatné funkce tréninkového kroku (jednu pro diskriminátor a jednu pro generátor). To jsme udělali záměrně, abychom izolovali tréninkové kroky pro každou síť, aby bylo jasnější, jak je každá síť trénována. Nicméně, tato implementace není efektivní, protože vyžaduje dvě samostatné volání generátoru pro každý tréninkový krok. V následující implementaci vytvoříme jedinou funkci `train_step()`, která provádí sekvenční trénink obou sítí. Také si všimněte, že vektor šumu je nyní 4-dimenzionální.

### 10.1 Funkce `train_step()`

In [ ]:
def train_step(real_images):
    optimizer_G.zero_grad()
    optimizer_D.zero_grad()
    # Create ground truth labels for real and fake data.
    labels = torch.cat(
        (
            torch.ones(TrainingConfig.BATCH_SIZE, 1), # Labels for real data
            torch.zeros(TrainingConfig.BATCH_SIZE, 1) # Labels for fake data
        ), dim=0
    ).to(TrainingConfig.DEVICE)

    # We want to fool the discriminator, so we create grond truth labels for
    # real images even though we feed the discriminator with fake images.
    misleading_labels = torch.ones(
        TrainingConfig.BATCH_SIZE, 1
    ).to(TrainingConfig.DEVICE)

    noise = torch.randn(
        TrainingConfig.BATCH_SIZE,
        TrainingConfig.LATENT_DIM,
        1, 1
    ).to(TrainingConfig.DEVICE)

    #----------------------------------------------------------------
    # First, generate fake images needed for training both networks.
    #----------------------------------------------------------------
    # Generate fake images.
    fake_images = generator(noise)

    # Predicted labels from discriminator for real images.
    y_pred_real = discriminator(real_images)

    # Predicted labels from discriminator for fake images.
    y_pred_fake = discriminator(fake_images)

    # Concatenate predictions for real and fake input data.
    y_pred_D = torch.cat((
        y_pred_real, y_pred_fake
    ), dim=0)

    # Compute the discriminator loss.
    D_loss = loss_func(labels, y_pred_D)

    # Discriminator step.
    D_loss.backward()
    optimizer_D.step()

    # Generate fake images.
    fake_images = generator(noise)

    # Predicted labels from discriminator for fake images.
    y_pred_fake = discriminator(fake_images)

    # Compute the generator loss.
    G_loss = loss_func(misleading_labels, y_pred_fake)

    # Generator step.
    G_loss.backward()
    optimizer_G.step()

    return D_loss, G_loss

### 10.2 Komfortní Funkce `plot_images()`

In [ ]:
# Create `BATCH_SIZE` fixed noise vector for generating examples images during training.
fixed_noise = torch.randn(
        TrainingConfig.BATCH_SIZE,
        TrainingConfig.LATENT_DIM,
        1, 1)

def plot_images(model):

    model.eval()
    with torch.no_grad():
        images = model(fixed_noise.to(TrainingConfig.DEVICE))
        images = images.detach().cpu()

    plt.figure(figsize=(20, 4))

    nrow = 2
    ncol = int(images.shape[0] / nrow)

    for i, image in enumerate(images):
        col = int(TrainingConfig.BATCH_SIZE / 2)
        plt.subplot(nrow, ncol, i+1)
        image = (image.permute(1, 2, 0).squeeze(0) + 1) * 127.5
        plt.imshow(np.array(image).astype('uint8'))
        plt.axis('off')

    plt.show();

### 10.3 Funkce `train()`

In [ ]:
G_LOSS = []
D_LOSS = []

def train(dataloader, epochs):
    for epoch in range(epochs):
        generator.train()
        discriminator.train()
        start = time.time()
        i = 0
        D_loss_list, G_loss_list = [], []

        for bi, data in tqdm(enumerate(dataloader), total=int(len(dataloader))):
            image_batch, _ = data
            image_batch = image_batch.to(TrainingConfig.DEVICE)

            i += 1

            D_loss, G_loss = train_step(image_batch)

            D_loss_list.append(D_loss)
            G_loss_list.append(G_loss)

        if (epoch + 1) % 5 == 0:
            torch.save(generator, generator_ckpt)
            torch.save(discriminator, discriminatr_ckpt)

        epoch_D_loss = sum(D_loss_list)/len(D_loss_list)
        epoch_G_loss = sum(G_loss_list)/len(G_loss_list)

        D_LOSS.append(epoch_D_loss.detach().cpu())
        G_LOSS.append(epoch_G_loss.detach().cpu())

        print('\n')
        print(f"Time for epoch {epoch + 1} is {time.time()-start} sec")
        print(f"Generator loss: {epoch_G_loss:.3f}, Discriminator loss: {epoch_D_loss:.3f}")

        plot_images(generator)

### 10.4 Trénink GAN sítě

In [ ]:
train(train_dataloader, 51)

## 11 Zobrazení grafů výsledků tréninku

In [ ]:
def plot_results(metrics, ylabel=None, ylim=None, metric_name=None, color=None):

    fig, ax = plt.subplots(figsize=(18, 5))

    if not (isinstance(metric_name, list) or isinstance(metric_name, tuple)):
        metrics = [metrics,]
        metric_name = [metric_name,]

    for idx, metric in enumerate(metrics):
        ax.plot(metric, color=color[idx])

    plt.xlabel("Epoch")
    plt.ylabel(ylabel)
    plt.title(ylabel)
    plt.xlim([0, TrainingConfig.NUM_EPOCHS-1])
    plt.ylim(ylim)
    # Tailor x-axis tick marks
    ax.xaxis.set_major_locator(MultipleLocator(5))
    ax.xaxis.set_major_formatter(FormatStrFormatter('%d'))
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    plt.grid(True)
    plt.legend(metric_name)
    plt.show(block=block_plot)

In [ ]:
# Retrieve the training results.
plot_results([G_LOSS, D_LOSS ],
            ylabel="Loss",
            ylim=[0, 5],
            metric_name=["Generator Loss", "Discriminator Loss"],
            color=["g", "b"]);

Z výše uvedených grafů vidíme, že na začátku je ztráta generátoru vysoká a ztráta diskriminátoru nízká. To dává smysl, protože generátor se snaží zjistit, jak vytvořit věrohodné obrázky, a diskriminátor nemá velké potíže s přesnými předpověďmi. Jakmile generátor začne věci chápat (i po pouhých 2-3 epochách), ztráta generátoru klesá. Všimli jsme si také mírného zvýšení ztráty diskriminátoru, protože má větší potíže s přesnými předpověďmi.

Ale pak, po přibližně pěti epochách, ztráta diskriminátoru velmi pozvolna klesá, ale ztráta generátoru začíná stoupat. To však nutně neznamená, že nejlepší obrázky byly vytvořeny po pouhých třech nebo čtyřech epochách. Pokud porovnáte ukázkové obrázky ve 30 epochách s těmi z období kolem 4-5 epoch, měli byste si všimnout celkově vyšší kvality po 30 epochách, i když se ztráta generátoru více než zdvojnásobila. Ve skutečnosti je to jeden z příkladů toho, proč může být trénink GAN obtížný, protože neexistuje jasně definované kritérium "zastavení" treninku, které by určovalo "nejlepší" model.


<img src="https://opencv.org/wp-content/uploads/2022/09/c4-gan-dcgan-train-comparison.png">

## 12 Vytváření obrázků pomocí natrénovaného modelu

In [ ]:
generator = Generator(TrainingConfig.LATENT_DIM)

In [ ]:
trained_generator = torch.load(generator_ckpt)

In [ ]:
print(trained_generator)

In [ ]:
def generate_images(model, noise):

    model.eval()
    with torch.no_grad():
        images = model(noise.to(TrainingConfig.DEVICE))
        images = images.detach().cpu()

    plt.figure(figsize=(20, 4))

    nrow = 2
    ncol = int(images.shape[0] / nrow)

    for i, image in enumerate(images):
        col = int(TrainingConfig.BATCH_SIZE / 2)
        plt.subplot(nrow, ncol, i+1)
        image = (image.permute(1, 2, 0).squeeze(0) + 1) * 127.5
        plt.imshow(np.array(image).astype('uint8'))
        plt.axis('off')

    plt.show();

In [ ]:
noise = torch.randn(
    TrainingConfig.BATCH_SIZE,
    TrainingConfig.LATENT_DIM,
    1, 1
).to(TrainingConfig.DEVICE)

In [ ]:
generate_images(trained_generator, noise)

## 13 Závěr

V tomto notebooku jsme se naučili implementovat a trénovat jednoduchou GAN síť pomocí CNN k generování tváří lidí. Některé z vygenerovaných obrázků výše vypadají docela dobře, zatímco jiné ne. S tak jednoduchým DCGAN modelem je však velmi obtížné jít dál. Existují pokročilejší a komplexnější architektury GAN sítí, které nyní dokážou generovat HD obrázky lidských tváří, které jsou nerozeznatelné od fotografií skutečných lidí.

In [ ]:
import shutil

def zip_folder_with_shutil(source_folder, output_path):
    '''Function for zip TensorBoard data'''
    shutil.make_archive(output_path, 'zip', source_folder)

In [ ]:
zip_folder_with_shutil('/kaggle/working/model_checkpoint', '/kaggle/working/model_checkpoint')